In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


In [4]:
train_df = pd.read_csv('./new_train.csv', parse_dates=["startdate"])
test_df = pd.read_csv('./new_test.csv', parse_dates=["startdate"])

In [5]:
exclude_cols = ['index', 'startdate']
temporal_attrs = ['year', 'month', 'day_of_year', 'season', 'day_of_year_sin', 'day_of_year_cos','month_sin', 'month_cos', 'season_sin', 'season_cos']
loc_attrs = ['lat', 'lon', 'loc_group']
embedding_attrs = ['climateregions__climateregion']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in temporal_attrs and c not in loc_attrs and c not in target and c not in embedding_attrs]
print("Main features:", len(main_attrs))

Main features: 240


In [6]:
train_independent_corr = train_df[main_attrs + temporal_attrs + loc_attrs + embedding_attrs].corr()
train_seleted_corr_columns = np.full((train_independent_corr.shape[0],), True, dtype=bool)
for i in range(train_independent_corr.shape[0]):
    for j in range(i + 1, train_independent_corr.shape[0]):
        if train_independent_corr.iloc[i, j] >= 0.96:
            
            if train_seleted_corr_columns[j]:
                train_seleted_corr_columns[j] = False
train_selected_columns = train_df[main_attrs + temporal_attrs + loc_attrs + embedding_attrs].columns[train_seleted_corr_columns]
print(train_selected_columns.shape)

(203,)


In [7]:
from sklearn.model_selection import KFold
import xgboost as xgb
from catboost import CatBoostRegressor
import lightgbm as lgb
from scipy.stats import gmean
X_train, X_validation, y_train, y_validation = train_test_split(train_df[train_selected_columns], train_df[target[0]], train_size=0.66, random_state=42)
categorical_features_indices = np.where(X_train.dtypes != float)[0]
categorical_feautures = [X_train.columns.tolist()[i] for i in categorical_features_indices]
X_test = test_df[train_selected_columns]
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_validation, y_validation, cat_features=categorical_features_indices)
clf = CatBoostRegressor(iterations=5000, verbose=200, task_type="GPU",
                           devices='0')
clf.fit(train_pool)

Learning rate set to 0.03182
0:	learn: 9.6014484	total: 37.9ms	remaining: 3m 9s
200:	learn: 1.5480178	total: 4.59s	remaining: 1m 49s
400:	learn: 1.2648072	total: 9.65s	remaining: 1m 50s
600:	learn: 1.0964234	total: 14.7s	remaining: 1m 47s
800:	learn: 0.9869025	total: 22.2s	remaining: 1m 56s
1000:	learn: 0.9036591	total: 27.4s	remaining: 1m 49s
1200:	learn: 0.8384113	total: 32.9s	remaining: 1m 44s
1400:	learn: 0.7865401	total: 37.6s	remaining: 1m 36s
1600:	learn: 0.7440282	total: 42.4s	remaining: 1m 29s
1800:	learn: 0.7072648	total: 46.9s	remaining: 1m 23s
2000:	learn: 0.6764907	total: 51.7s	remaining: 1m 17s
2200:	learn: 0.6484618	total: 56.4s	remaining: 1m 11s
2400:	learn: 0.6237881	total: 1m 1s	remaining: 1m 6s
2600:	learn: 0.6022826	total: 1m 5s	remaining: 1m
2800:	learn: 0.5837379	total: 1m 10s	remaining: 55.5s
3000:	learn: 0.5657454	total: 1m 15s	remaining: 50.4s
3200:	learn: 0.5500055	total: 1m 21s	remaining: 45.9s
3400:	learn: 0.5356226	total: 1m 26s	remaining: 40.8s
3600:	learn

In [8]:
feature_importances = clf.get_feature_importance(train_pool)
feature_names = train_selected_columns
selected_cols = []
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    if score >= 0.06:
        selected_cols.append(name)
    print('{}: {}'.format(name, score))

nmme-tmp2m-56w__cancm3: 25.621289322161253
contest-wind-h500-14d__wind-hgt-500: 13.321094256638402
contest-pevpr-sfc-gauss-14d__pevpr: 8.812317387454616
contest-slp-14d__slp: 5.461737782969856
nmme-tmp2m-34w__ccsm3: 5.362613711483253
elevation__elevation: 4.773468170717297
lon: 4.546033073423309
contest-prwtr-eatm-14d__prwtr: 4.3999783292950365
lat: 4.3446488950809234
climateregions__climateregion: 3.150926289819827
contest-wind-h100-14d__wind-hgt-100: 3.031810223895437
contest-pres-sfc-gauss-14d__pres: 2.9659074555961613
season_sin: 2.032815038215516
month_cos: 0.7219874381072938
day_of_year_sin: 0.7172472478749553
icec-2010-9: 0.595974233641957
contest-precip-14d__precip: 0.491070758434198
contest-wind-uwnd-250-14d__wind-uwnd-250: 0.41609417407472893
nmme-prate-34w__cfsv2: 0.3692048423205802
nmme-prate-34w__nasa: 0.34014323382279055
icec-2010-4: 0.3088065928735785
nmme-prate-56w__gfdlflora: 0.3017898724124015
wind-uwnd-250-2010-7: 0.26111028969887945
contest-wind-vwnd-925-14d__wind-v

In [9]:
out_cols = ['icec-2010-1',
 'icec-2010-2',
 'icec-2010-3',
 'icec-2010-4',
 'icec-2010-5',
 'icec-2010-6',
 'icec-2010-7',
 'icec-2010-8',
 'icec-2010-9',
 'icec-2010-10',
 'year',
 'quarter',
 'month_cos',
 'quarter_sin',
 'quarter_cos']
selected_cols = [c for c in selected_cols if c not in out_cols]
len(selected_cols)

65

In [10]:
i_X_train, i_X_validation, i_y_train, i_y_validation = train_test_split(train_df[selected_cols], train_df[target[0]], train_size=0.66, random_state=42)
i_categorical_features_indices = np.where(i_X_train.dtypes != float)[0]
i_categorical_feautures = [i_X_train.columns.tolist()[i] for i in i_categorical_features_indices]
i_X_test = test_df[selected_cols]
i_train_pool = Pool(i_X_train, i_y_train, cat_features=i_categorical_features_indices)
i_validate_pool = Pool(i_X_validation, i_y_validation, cat_features=i_categorical_features_indices)
i_clf = CatBoostRegressor(iterations=5000, verbose=200, task_type="GPU",
                           devices='0')
i_clf.fit(i_train_pool)

Learning rate set to 0.03182
0:	learn: 9.6007322	total: 12ms	remaining: 1m
200:	learn: 1.5549379	total: 2.96s	remaining: 1m 10s
400:	learn: 1.2701139	total: 5.91s	remaining: 1m 7s
600:	learn: 1.1054168	total: 9.79s	remaining: 1m 11s
800:	learn: 0.9895309	total: 12.4s	remaining: 1m 5s
1000:	learn: 0.9058183	total: 15s	remaining: 1m
1200:	learn: 0.8429471	total: 17.5s	remaining: 55.3s
1400:	learn: 0.7904880	total: 20.1s	remaining: 51.7s
1600:	learn: 0.7476281	total: 22.7s	remaining: 48.1s
1800:	learn: 0.7105435	total: 25.2s	remaining: 44.8s
2000:	learn: 0.6798694	total: 27.9s	remaining: 41.8s
2200:	learn: 0.6536556	total: 30.5s	remaining: 38.8s
2400:	learn: 0.6288278	total: 33s	remaining: 35.7s
2600:	learn: 0.6073853	total: 35.8s	remaining: 33s
2800:	learn: 0.5888877	total: 39s	remaining: 30.6s
3000:	learn: 0.5713476	total: 41.8s	remaining: 27.9s
3200:	learn: 0.5552397	total: 44.6s	remaining: 25.1s
3400:	learn: 0.5406028	total: 47.5s	remaining: 22.3s
3600:	learn: 0.5275001	total: 50.4s	r

In [12]:
diff_cols = ['contest-pevpr-sfc-gauss-14d__pevpr', 'nmme0-tmp2m-34w__cancm30', 'year']
diff_selected_cols = [c for c in selected_cols if c not in diff_cols]
len(diff_selected_cols)

63

In [15]:
diff_selected_cols

['nmme-tmp2m-56w__cancm3',
 'contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-wind-h100-14d__wind-hgt-100',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'month_sin',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2

In [13]:
X = train_df[diff_selected_cols].values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=5000, verbose=200, task_type="GPU",
                           devices='0:1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=True, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)

5it [00:14,  2.91s/it]


(375734, 63) (375734,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]Warning: less than 75% gpu memory available for training. Free: 7458.5625 Total: 24268.3125


Learning rate set to 0.072699
0:	learn: 9.2499207	test: 9.2284898	best: 9.2284898 (0)	total: 158ms	remaining: 13m 7s
1:	learn: 8.6819595	test: 8.6627952	best: 8.6627952 (1)	total: 258ms	remaining: 10m 43s
2:	learn: 8.1555965	test: 8.1371740	best: 8.1371740 (2)	total: 318ms	remaining: 8m 49s
3:	learn: 7.6667561	test: 7.6504468	best: 7.6504468 (3)	total: 334ms	remaining: 6m 57s
4:	learn: 7.2170752	test: 7.2011103	best: 7.2011103 (4)	total: 378ms	remaining: 6m 17s
5:	learn: 6.8031586	test: 6.7895230	best: 6.7895230 (5)	total: 477ms	remaining: 6m 36s
6:	learn: 6.4227982	test: 6.4109669	best: 6.4109669 (6)	total: 548ms	remaining: 6m 31s
7:	learn: 6.0711788	test: 6.0604547	best: 6.0604547 (7)	total: 607ms	remaining: 6m 19s
8:	learn: 5.7410935	test: 5.7307136	best: 5.7307136 (8)	total: 693ms	remaining: 6m 24s
9:	learn: 5.4405190	test: 5.4309492	best: 5.4309492 (9)	total: 816ms	remaining: 6m 46s
10:	learn: 5.1658073	test: 5.1568380	best: 5.1568380 (10)	total: 930ms	remaining: 7m 1s
11:	learn: 

100:	learn: 1.5008054	test: 1.5014044	best: 1.5014044 (100)	total: 1.87s	remaining: 1m 30s
101:	learn: 1.4963696	test: 1.4969672	best: 1.4969672 (101)	total: 1.87s	remaining: 1m 30s
102:	learn: 1.4926467	test: 1.4934982	best: 1.4934982 (102)	total: 1.88s	remaining: 1m 29s
103:	learn: 1.4866115	test: 1.4875784	best: 1.4875784 (103)	total: 1.88s	remaining: 1m 28s
104:	learn: 1.4825408	test: 1.4836421	best: 1.4836421 (104)	total: 1.89s	remaining: 1m 28s
105:	learn: 1.4786231	test: 1.4797883	best: 1.4797883 (105)	total: 1.9s	remaining: 1m 27s
106:	learn: 1.4743675	test: 1.4755140	best: 1.4755140 (106)	total: 1.91s	remaining: 1m 27s
107:	learn: 1.4702092	test: 1.4713393	best: 1.4713393 (107)	total: 1.91s	remaining: 1m 26s
108:	learn: 1.4665170	test: 1.4677339	best: 1.4677339 (108)	total: 1.91s	remaining: 1m 25s
109:	learn: 1.4631203	test: 1.4643258	best: 1.4643258 (109)	total: 1.92s	remaining: 1m 25s
110:	learn: 1.4577569	test: 1.4591558	best: 1.4591558 (110)	total: 1.92s	remaining: 1m 24s


...

4604:	learn: 0.3375017	test: 0.3645662	best: 0.3645662 (4604)	total: 1m 18s	remaining: 6.69s
4605:	learn: 0.3374626	test: 0.3645335	best: 0.3645335 (4605)	total: 1m 18s	remaining: 6.67s
4606:	learn: 0.3374299	test: 0.3645020	best: 0.3645020 (4606)	total: 1m 18s	remaining: 6.66s
4607:	learn: 0.3373850	test: 0.3644518	best: 0.3644518 (4607)	total: 1m 18s	remaining: 6.64s
4608:	learn: 0.3373605	test: 0.3644235	best: 0.3644235 (4608)	total: 1m 18s	remaining: 6.62s
4609:	learn: 0.3373163	test: 0.3643733	best: 0.3643733 (4609)	total: 1m 18s	remaining: 6.6s
4610:	learn: 0.3372878	test: 0.3643478	best: 0.3643478 (4610)	total: 1m 18s	remaining: 6.58s
4611:	learn: 0.3372567	test: 0.3643211	best: 0.3643211 (4611)	total: 1m 18s	remaining: 6.57s
4612:	learn: 0.3372350	test: 0.3643047	best: 0.3643047 (4612)	total: 1m 18s	remaining: 6.55s
4613:	learn: 0.3371999	test: 0.3642692	best: 0.3642692 (4613)	total: 1m 18s	remaining: 6.53s
4614:	learn: 0.3371604	test: 0.3642311	best: 0.3642311 (4614)	total: 1m

4713:	learn: 0.3338869	test: 0.3612172	best: 0.3612172 (4713)	total: 1m 20s	remaining: 4.86s
4714:	learn: 0.3338581	test: 0.3611825	best: 0.3611825 (4714)	total: 1m 20s	remaining: 4.84s
4715:	learn: 0.3338276	test: 0.3611520	best: 0.3611520 (4715)	total: 1m 20s	remaining: 4.82s
4716:	learn: 0.3337882	test: 0.3611127	best: 0.3611127 (4716)	total: 1m 20s	remaining: 4.8s
4717:	learn: 0.3337467	test: 0.3610743	best: 0.3610743 (4717)	total: 1m 20s	remaining: 4.78s
4718:	learn: 0.3337245	test: 0.3610587	best: 0.3610587 (4718)	total: 1m 20s	remaining: 4.77s
4719:	learn: 0.3337031	test: 0.3610404	best: 0.3610404 (4719)	total: 1m 20s	remaining: 4.75s
4720:	learn: 0.3336779	test: 0.3610164	best: 0.3610164 (4720)	total: 1m 20s	remaining: 4.74s
4721:	learn: 0.3336428	test: 0.3609889	best: 0.3609889 (4721)	total: 1m 20s	remaining: 4.72s
4722:	learn: 0.3336237	test: 0.3609725	best: 0.3609725 (4722)	total: 1m 20s	remaining: 4.71s
4723:	learn: 0.3335266	test: 0.3608814	best: 0.3608814 (4723)	total: 1m

4809:	learn: 0.3307885	test: 0.3583240	best: 0.3583240 (4809)	total: 1m 21s	remaining: 3.21s
4810:	learn: 0.3307619	test: 0.3583030	best: 0.3583030 (4810)	total: 1m 21s	remaining: 3.19s
4811:	learn: 0.3307355	test: 0.3582826	best: 0.3582826 (4811)	total: 1m 21s	remaining: 3.18s
4812:	learn: 0.3307057	test: 0.3582537	best: 0.3582537 (4812)	total: 1m 21s	remaining: 3.16s
4813:	learn: 0.3306740	test: 0.3582203	best: 0.3582203 (4813)	total: 1m 21s	remaining: 3.14s
4814:	learn: 0.3306501	test: 0.3581956	best: 0.3581956 (4814)	total: 1m 21s	remaining: 3.12s
4815:	learn: 0.3306237	test: 0.3581711	best: 0.3581711 (4815)	total: 1m 21s	remaining: 3.11s
4816:	learn: 0.3305974	test: 0.3581464	best: 0.3581464 (4816)	total: 1m 21s	remaining: 3.09s
4817:	learn: 0.3305553	test: 0.3581096	best: 0.3581096 (4817)	total: 1m 21s	remaining: 3.07s
4818:	learn: 0.3305221	test: 0.3580816	best: 0.3580816 (4818)	total: 1m 21s	remaining: 3.06s
4819:	learn: 0.3304870	test: 0.3580434	best: 0.3580434 (4819)	total: 1

4927:	learn: 0.3271040	test: 0.3549346	best: 0.3549346 (4927)	total: 1m 22s	remaining: 1.2s
4928:	learn: 0.3270776	test: 0.3549124	best: 0.3549124 (4928)	total: 1m 22s	remaining: 1.19s
4929:	learn: 0.3270569	test: 0.3548920	best: 0.3548920 (4929)	total: 1m 22s	remaining: 1.17s
4930:	learn: 0.3270364	test: 0.3548744	best: 0.3548744 (4930)	total: 1m 22s	remaining: 1.15s
4931:	learn: 0.3270144	test: 0.3548559	best: 0.3548559 (4931)	total: 1m 22s	remaining: 1.14s
4932:	learn: 0.3269771	test: 0.3548144	best: 0.3548144 (4932)	total: 1m 22s	remaining: 1.12s
4933:	learn: 0.3269538	test: 0.3548012	best: 0.3548012 (4933)	total: 1m 22s	remaining: 1.1s
4934:	learn: 0.3269208	test: 0.3547619	best: 0.3547619 (4934)	total: 1m 22s	remaining: 1.09s
4935:	learn: 0.3268871	test: 0.3547354	best: 0.3547354 (4935)	total: 1m 22s	remaining: 1.07s
4936:	learn: 0.3268653	test: 0.3547212	best: 0.3547212 (4936)	total: 1m 22s	remaining: 1.05s
4937:	learn: 0.3267995	test: 0.3546553	best: 0.3546553 (4937)	total: 1m 

100%|███████████████████████████████████████████████████████████████████████████████████████| 5/5 [10:04<00:00, 120.98s/it]


In [14]:
preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test_df[diff_selected_cols].values))
s = 0
for pred in preds:
    s += pred
s = s/5
df = pd.read_csv('./sample_solution.csv')
df['contest-tmp2m-14d__tmp2m'] = s
df.to_csv('submission_corr_important_v3.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  8.60it/s]


cos_sim with best submission: 0.9993133784518393
